In [80]:
%load_ext autoreload
%autoreload 2
import numpy as np
import qutip
import theano
import theano.tensor as T
import QubitNetwork

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
net2 = QubitNetwork.load_network_from_file('netdump.pickle')
net2.J.get_value()

array([ 0.66994226,  0.01365408, -0.19861324,  0.19601662,  1.19182957,
        1.21698788,  1.71885772, -1.26862474, -0.88005872,  0.15469832,
        0.27293408,  0.52832768,  0.65341716,  0.72084536,  0.17985858,
        0.08892903, -0.52529778,  1.19547534,  1.14115972,  2.30174355])

In [74]:
# theano.config.optimizer = 'None'
net = QubitNetwork.QubitNetwork(num_qubits=4,
                           interactions=('all', ['xx', 'yy', 'zz']),
                           self_interactions=('all', ['x', 'y', 'z']),
                           system_qubits=[0, 1, 2])
QubitNetwork.sgd_optimization(net=net, learning_rate=.01, batch_size=20)

Building the model...
Let's roll!
Epoch 0, 0.06014634115021243
Epoch 1, 0.06054712969329356
Epoch 2, 0.060926802772246776
Epoch 3, 0.06128502392013427
Epoch 4, 0.06162181513169072
Epoch 5, 0.0619375100227286
Epoch 6, 0.06223270233829549
Epoch 7, 0.06250819331087351
Epoch 8, 0.06276494071996612
Epoch 9, 0.06300401177173963
Finished training


In [76]:
QubitNetwork.sgd_optimization(
    net='net_on_training.pickle',
    learning_rate=0.01,
    n_epochs=100,
    batch_size=20
)

Building the model...
Let's roll!
Epoch 0, 0.06932299772490925
Epoch 1, 0.06935318163692862
Epoch 2, 0.06938421649236837
Epoch 3, 0.06941601028113345
Epoch 4, 0.06944847035243532
Epoch 5, 0.06948150431793762
Epoch 6, 0.06951502082760051
Epoch 7, 0.06954893022986748
Epoch 8, 0.06958314512727831
Epoch 9, 0.06961758083796352
Epoch 10, 0.06965215577281739
Epoch 11, 0.06968679173746822
Epoch 12, 0.06972141416749116
Epoch 13, 0.06975595230464804
Epoch 14, 0.06979033932129713
Epoch 15, 0.069824512399502
Epoch 16, 0.06985841277078689
Epoch 17, 0.06989198572193561
Epoch 18, 0.06992518057171164
Epoch 19, 0.06995795062290018
Epoch 20, 0.06999025309361975
Epoch 21, 0.07002204903143695
Epoch 22, 0.07005330321343722
Epoch 23, 0.07008398403504495
Epoch 24, 0.07011406339006926
Epoch 25, 0.07014351654415096
Epoch 26, 0.07017232200351584
Epoch 27, 0.07020046138069522
Epoch 28, 0.07022791925864955
Epoch 29, 0.07025468305452763
Epoch 30, 0.07028074288411529
Epoch 31, 0.0703060914278581
Epoch 32, 0.0703307

In [81]:
QubitNetwork.sgd_optimization(
    net='net_on_training.pickle',
    learning_rate=0.01,
    n_epochs=200,
    batch_size=100
)

Building the model...
Let's roll!
Epoch 0, 0.0837915662754077
Epoch 1, 0.08380147864921984
Epoch 2, 0.08381131076517158
Epoch 3, 0.08382106794206369
Epoch 4, 0.08383075498405197
Epoch 5, 0.0838403762294213
Epoch 6, 0.08384993559487382
Epoch 7, 0.08385943661574151
Epoch 8, 0.08386888248248452
Epoch 9, 0.08387827607381455
Epoch 10, 0.08388761998675107
Epoch 11, 0.08389691656388458
Epoch 12, 0.08390616791810356
Epoch 13, 0.08391537595501343
Epoch 14, 0.08392454239325996
Epoch 15, 0.08393366878294724
Epoch 16, 0.08394275652232389
Epoch 17, 0.08395180687289602
Epoch 18, 0.08396082097311304
Epoch 19, 0.08396979985075247
Epoch 20, 0.08397874443413025
Epoch 21, 0.08398765556223721
Epoch 22, 0.0839965339939069
Epoch 23, 0.08400538041610159
Epoch 24, 0.08401419545139496
Epoch 25, 0.08402297966473549
Epoch 26, 0.08403173356954402
Epoch 27, 0.08404045763322088
Epoch 28, 0.0840491522821065
Epoch 29, 0.0840578179059563
Epoch 30, 0.08406645486196636
Epoch 31, 0.08407506347839998
Epoch 32, 0.084083644